In [1]:
import requests
import pandas as pd

# ---------------------------------------------------------
# 1. LOAD ALL CHARACTERS
# ---------------------------------------------------------
def get_all_characters():
    url = "https://rickandmortyapi.com/api/character"
    characters = []

    while url:
        response = requests.get(url).json()
        for c in response["results"]:
            characters.append({
                "id": c["id"],
                "name": c["name"],
                "status": c["status"],
                "species": c["species"],
                "gender": c["gender"],
                "origin": c["origin"]["name"],
                "location": c["location"]["name"],
                "episode_urls": c["episode"]  # list of episode URLs
            })
        url = response["info"]["next"]

    return pd.DataFrame(characters)


# ---------------------------------------------------------
# 2. LOAD ALL EPISODES
# ---------------------------------------------------------
def get_all_episodes():
    url = "https://rickandmortyapi.com/api/episode"
    episodes = []

    while url:
        response = requests.get(url).json()
        for e in response["results"]:
            episodes.append({
                "id": e["id"],
                "name": e["name"],
                "episode_code": e["episode"],
                "air_date": e["air_date"],
                "character_urls": e["characters"]
            })
        url = response["info"]["next"]

    return pd.DataFrame(episodes)


# ---------------------------------------------------------
# 3. BUILD CHARACTER–EPISODE BRIDGE TABLE
# ---------------------------------------------------------
def build_character_episode_bridge(characters_df):
    rows = []

    for _, row in characters_df.iterrows():
        char_id = row["id"]
        for ep_url in row["episode_urls"]:
            ep_id = int(ep_url.split("/")[-1])  # extract episode ID
            rows.append({"character_id": char_id, "episode_id": ep_id})

    return pd.DataFrame(rows)


# ---------------------------------------------------------
# 4. COUNT EPISODES PER CHARACTER
# ---------------------------------------------------------
def count_episodes_per_character(character_episode_df):
    return (
        character_episode_df
        .groupby("character_id")
        .size()
        .reset_index(name="episode_count")
    )


# ---------------------------------------------------------
# 5. MERGE COUNTS BACK INTO CHARACTERS TABLE
# ---------------------------------------------------------
def merge_character_episode_counts(characters_df, episode_counts_df):
    merged = characters_df.merge(
        episode_counts_df,
        left_on="id",
        right_on="character_id",
        how="left"
    )
    return merged.drop(columns=["character_id"])


# ---------------------------------------------------------
# RUN THE FULL PIPELINE
# ---------------------------------------------------------
characters_df = get_all_characters()
episodes_df = get_all_episodes()
character_episode_df = build_character_episode_bridge(characters_df)
episode_counts_df = count_episodes_per_character(character_episode_df)
final_df = merge_character_episode_counts(characters_df, episode_counts_df)

# ---------------------------------------------------------
# SHOW RESULT
# ---------------------------------------------------------
print(final_df.head())
print(f"\nTotal characters: {len(final_df)}")


   id          name status species  gender                         origin  \
0   1  Rick Sanchez  Alive   Human    Male                  Earth (C-137)   
1   2   Morty Smith  Alive   Human    Male                        unknown   
2   3  Summer Smith  Alive   Human  Female  Earth (Replacement Dimension)   
3   4    Beth Smith  Alive   Human  Female  Earth (Replacement Dimension)   
4   5   Jerry Smith  Alive   Human    Male  Earth (Replacement Dimension)   

                        location  \
0               Citadel of Ricks   
1               Citadel of Ricks   
2  Earth (Replacement Dimension)   
3  Earth (Replacement Dimension)   
4  Earth (Replacement Dimension)   

                                        episode_urls  episode_count  
0  [https://rickandmortyapi.com/api/episode/1, ht...             51  
1  [https://rickandmortyapi.com/api/episode/1, ht...             51  
2  [https://rickandmortyapi.com/api/episode/6, ht...             42  
3  [https://rickandmortyapi.com/api/epis